## Import libary-library yang dibutuhkan

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from skfuzzy.control import Rule

## 1. Fuzifikasi

### Input data

In [2]:
df = pd.read_excel('./data/brain_stroke.xlsx')
df.head()

,age,avg_glucose_level,bmi,cigarettes per day,Class stroke
0,44,89.30,21.4,16,0
1,69,123.21,15.1,29,0
2,55,92.48,18.0,10,0
3,63,79.89,31.6,6,0
4,70,73.62,20.8,15,0


In [3]:
# Input variable
age = ctrl.Antecedent(np.arange(20, 71, 1), 'age')
glucose = ctrl.Antecedent(np.arange(40, 121, 1), 'glucose')
bmi = ctrl.Antecedent(np.arange(10, 61, 1), 'bmi')
cigarette = ctrl.Antecedent(np.arange(0, 51, 1), 'cigarette')

# Output variable
stroke_risk = ctrl.Consequent(np.arange(0, 101, 1), 'stroke_risk')

### Mendefinisikan Fungsi Keanggotaan

In [4]:
# Age
age['young'] = fuzz.trimf(age.universe, [25, 35, 45])
age['adult'] = fuzz.trimf(age.universe, [35, 45, 55])
age['old'] = fuzz.trimf(age.universe, [45, 55, 65])

In [5]:
# Glukosa
glucose['low'] = fuzz.trimf(glucose.universe, [50, 70, 90])
glucose['high'] = fuzz.trimf(glucose.universe, [60, 80, 100])

In [6]:
# BMI
bmi['min'] = fuzz.trimf(bmi.universe, [10, 20, 30])
bmi['mid'] = fuzz.trimf(bmi.universe, [20, 30, 40])
bmi['high'] = fuzz.trimf(bmi.universe, [30, 40, 50])

In [7]:
# Cigarette
cigarette['low'] = fuzz.trimf(cigarette.universe, [5, 17.5, 30])
cigarette['high'] = fuzz.trimf(cigarette.universe, [15, 30, 45])

In [8]:
# OUTPUT
stroke_risk['low'] = fuzz.trimf(stroke_risk.universe, [0, 0, 50])
stroke_risk['high'] = fuzz.trimf(stroke_risk.universe, [50, 100, 100])

## 2. Inferensi

### Basis Pengetahuan

In [9]:
rules =  [
    Rule(age['old'] & bmi['high'] & cigarette['high'] & glucose['high'], stroke_risk['high']),
    Rule(age['old'] & bmi['mid'] & cigarette['high'] & glucose['high'], stroke_risk['high']),
    Rule(age['adult'] & bmi['high'] & cigarette['high'] & glucose['high'], stroke_risk['high']),
    Rule(age['old'] & bmi['mid'] & cigarette['low'] & glucose['high'], stroke_risk['high']),
    
    Rule(age['adult'] & bmi['mid'] & cigarette['low'] & glucose['low'], stroke_risk['low']),
    Rule(age['old'] & bmi['mid'] & cigarette['low'] & glucose['low'], stroke_risk['low']),
    Rule(age['adult'] & bmi['mid'] & cigarette['high'] & glucose['low'], stroke_risk['high']),
    Rule(age['young'] & bmi['high'] & cigarette['high'] & glucose['high'], stroke_risk['high']),
    Rule(age['young'] & bmi['high'] & cigarette['low'] & glucose['high'], stroke_risk['high']),
    Rule(age['young'] & bmi['mid'] & cigarette['high'] & glucose['high'], stroke_risk['high']),
    
    Rule(age['young'] & bmi['mid'] & cigarette['low'] & glucose['low'], stroke_risk['low']),
    Rule(age['young'] & bmi['min'] & cigarette['low'] & glucose['low'], stroke_risk['low']),
    Rule(age['adult'] & bmi['min'] & cigarette['low'] & glucose['low'], stroke_risk['low']),
    Rule(age['young'] & bmi['high'] & cigarette['low'] & glucose['low'], stroke_risk['low']),
    Rule(age['young'] & bmi['min'] & cigarette['high'] & glucose['low'], stroke_risk['low']),
]

In [10]:
stroke_ctrl = ctrl.ControlSystem(rules)
stroke_sim = ctrl.ControlSystemSimulation(stroke_ctrl)

In [11]:
# Ubah nilai-nilainya di sini
stroke_sim.input['age'] = 38  
stroke_sim.input['glucose'] = 73.74
stroke_sim.input['bmi'] = 16
stroke_sim.input['cigarette'] = 22

## 3. Defuzzifikasi

In [12]:
stroke_sim.compute()
hasil = stroke_sim.output.get('stroke_risk', None)
if hasil is not None:
    print(f"Hasil seseorang mengalami stroke: {hasil:.2f}%")
    print(f'Hasil seseorang tidak mengalami stroke: {(100-hasil):.2f}%')
else:
    print("Stroke risk tidak terhitung karena tidak ada rule yang aktif.")

Hasil seseorang mengalami stroke: 18.57%
Hasil seseorang tidak mengalami stroke: 81.43%


## Baris-Baris yang dapat Digunakan untuk Menjalankan Program

In [13]:
# Fungsi pengecekan
def is_processable(age_val, glucose_val, bmi_val, cig_val):
    try:
        stroke_sim.input['age'] = age_val
        stroke_sim.input['glucose'] = glucose_val
        stroke_sim.input['bmi'] = bmi_val
        stroke_sim.input['cigarette'] = cig_val
        stroke_sim.compute()
        _ = stroke_sim.output['stroke_risk']
        return True
    except:
        return False

# Tandai baris yang bisa diproses
df['processable'] = df.apply(lambda row: is_processable(
    row['age'],
    row['avg_glucose_level'],
    row['bmi'],
    row['cigarettes per day']
), axis=1)

# Tampilkan hanya data yang bisa diproses
processable_data = df[df['processable']]
processable_data.head(25) # Jika ingin mencoba data yang lebih sedikit, cukup dengan mengubah argumennya

,age,avg_glucose_level,bmi,cigarettes per day,Class stroke,processable
0,44,89.30,21.4,16,0,True
3,63,79.89,31.6,6,0,True
10,38,73.74,16.0,22,0,True
11,26,73.20,16.8,25,0,True
21,40,84.59,17.7,22,0,True
23,34,86.11,19.0,13,0,True
33,42,70.78,20.3,18,0,True
36,47,64.06,18.9,15,0,True
41,39,86.75,22.3,27,0,True
43,36,78.05,25.7,20,0,True
